In [ ]:
import cv2, numpy as np
import pandas as pd

def concat(img1, img2)
  gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
  gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  # ORB, BF-Hamming 로 knnMatch  ---①
  detector = cv2.ORB_create()
  kp1, desc1 = detector.detectAndCompute(gray1, None)
  kp2, desc2 = detector.detectAndCompute(gray2, None)
  matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
  matches = matcher.match(desc1, desc2)

  # 매칭 결과를 거리기준 오름차순으로 정렬 ---③
  matches = sorted(matches, key=lambda x:x.distance)
  # for match in matches:
  #     print(f"img1:{kp1[match.queryIdx].pt} - img2:{kp2[match.queryIdx].pt}")
    
  # 모든 매칭점 그리기 ---④
  res1 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

  # 매칭점으로 원근 변환 및 영역 표시 ---⑤
  src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ])
  dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ])

  # RANSAC으로 변환 행렬 근사 계산 ---⑥
  mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

  h,w = img1.shape[:2]
  pts = np.float32([ [[0,0]],[[0,h-1]],[[w-1,h-1]],[[w-1,0]] ])
  dst = cv2.perspectiveTransform(pts,mtrx)
  #img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)

  # 정상치 매칭만 그리기 ---⑦
  matchesMask = mask.ravel().tolist()
  res2 = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, \
                      matchesMask = matchesMask,
                      flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
  # 모든 매칭점과 정상치 비율 ---⑧
  accuracy=float(mask.sum()) / mask.size
  print("accuracy: %d/%d(%.2f%%)"% (mask.sum(), mask.size, accuracy))

  # warped_image = cv2.warpPerspective(img1, mtrx, (img2.shape[1], img2.shape[0]))
  # cv2.imwrite("test.jpg", warped_image)

  #정상치 매칭 키포인트 리스트
  kp1_list = []
  kp2_list = []
  i = 0

  # 정상치 매칭 키포인트 저장
  while i < len(matchesMask):
      if matchesMask[i] == 1:
          print(f"img1:{kp1[matches[i].queryIdx].pt} - img2:{kp2[matches[i].queryIdx].pt}")
          kp1_list.append(kp1[matches[i].queryIdx])
          kp2_list.append(kp2[matches[i].queryIdx])
      i = i + 1

  # vector -> point 자료형 변경

  kp1_tuple_list = []
  kp2_tuple_list = []

  for k in kp1_list:
    kp1_tuple_list.append(k.pt)
    
  for k in kp2_list:
    kp2_tuple_list.append(k.pt)

  # padas dataFrame 변환
  df1 = pd.DataFrame(kp1_tuple_list, columns=['x','y'])
  df2 = pd.DataFrame(kp2_tuple_list, columns=['x','y'])

  # x 최대값 추출
  roi_point1=df1['x'].max()
  roi_point2=df2['x'].max()

  # 이미지 roi
  h1,w1,c1 = img1.shape
  h2,w2,c2 = img2.shape
  roi1 = img1[0:h1,0:int(roi_point1)]
  roi2 = img2[0:h2,int(roi_point2):w2]

  # 이미지 concat
  result = np.concatenate([roi1,roi2], axis = 0)
  return result 
  # 결과 출력                    
  
  #from google.colab.patches import cv2_imshow      

  #cv2_imshow(result)
  #cv2.waitKey()
  #cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import os


dir ="./"
img_list = []
combine_image = None

# 디렉토리 내부 사진 읽어오기
for f in os.listdir(dir):
  path = dir + f
  img_list.append(cv2.imread(path))

i = 0
while i < len(img_list) - 1:
  combine_img = combine(k[i],k[i+1])
  i +=1

cv2.imwrite("combine.jpg",combine)